# Capstone Project

### About

This notebook will be mainly used for the IBM capstone project

In [2]:
# import libraries
import pandas as pd
import numpy as np

In [3]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!
